In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime


# Reading an Excel file located at the specified path into a DataFrame
df=pd.read_excel("C:/Users/Praveen Kumar/Desktop/New data dump.xlsx")


# Converting the 'Release Date' column in the DataFrame to date-only format
df['Release Date'] = df['Release Date'].dt.date


# Getting the current date using the 'date.today()' method
today_date = date.today()


# Calculating the age of each entry in the DataFrame based on the 'Release Date' column
df['Age'] = ((today_date - df['Release Date']).dt.days / 365).round(1)


# The total sales is computed by adding the product of 'PO Buy Count' and 'PO Buy Avg'
# with the product of 'New Buy Count' and 'New Buy Avg', and rounding the result
df['Total Sales']= ((df['PO Buy Count']*df['PO Buy Avg']) + 
                    (df['New Buy Count']*df['New Buy Avg'])).round()


# Calculating the average of the 'Total Sales' column for entries where 'Total Sales' is not equal to zero
Average_of_total_sales= df[df['Total Sales'] !=0]['Total Sales'].mean()


# Calculating the percentage deviation of each entry's 'Total Sales'
df['% Deviation']=(df['Total Sales'] -Average_of_total_sales)/Average_of_total_sales


# Initialize lists to store the calculated values for different columns
arr = []
cname = []
hdays = []

# Loop through each entry in the DataFrame
for i in range(len(df)):
    if df['Category'][i] == "Nintendo Switch" and df['Page Visits'][i] >= 100:
        arr.append(5)
        cname.append('NS Games')
        hdays.append('<45 Days')
        
    elif df['Category'][i] == "Nintendo Switch":
        arr.append(6)
        cname.append('NS Junk')
        hdays.append('<45 Days')
        
    elif df['Ti-Credits'][i] >= 1900 or df['Pre Order'][i] == 'Yes':
        arr.append(1)
        cname.append('Volatile Games')
        hdays.append('<15 Days')
        
    elif df['% Deviation'][i] >= 0.80:
        arr.append(2)
        cname.append('All Time Hits')
        hdays.append('<30 Days')
                   
    elif df['% Deviation'][i] < 0.80 and df['% Deviation'][i] >= -0.40:
        arr.append(3)
        cname.append('Moderate')
        hdays.append('<30 Days')
        
    elif df['% Deviation'][i] < -0.40 and df['% Deviation'][i] >= -0.74:
        arr.append(4)
        cname.append('Low')
        hdays.append('<30 Days')
        
    elif df['Total Sales'][i] != 0 and (df['PO Inventory'][i] + df['New Inventory'][i]) > 0:
        arr.append(7)
        cname.append('Junk A')
        hdays.append('<45 Days')
    
    elif df['Total Sales'][i] == 0 and (df['PO Inventory'][i] + df['New Inventory'][i]) > 0:
        arr.append(8)
        cname.append('Junk B')
        hdays.append('Clearance')        

    elif df['Allow Sell'][i] == 'Yes' and (df['PO Inventory'][i] + df['New Inventory'][i]) == 0:
        arr.append(9)
        cname.append('Junk C')
        hdays.append('Clearance')
        
    elif df['Allow Sell'][i] == 'No':
        arr.append(10)
        cname.append('Junk D')
        hdays.append('Clearance')
        
    else:
        arr.append(0)
        cname.append(0)
        hdays.append(0)

# Adding the calculated values to each column in the DataFrame        
df['Rank'] = arr
df['Class Name'] = cname
df['Inv Hold Days'] = hdays      

# Calculating the cash margin for each entry in the DataFrame
CM = []  

for i in range(len(df)):
    try:
        if df['PO Price'][i] != 0 and not np.isnan(df['PO Price'][i]):
            relative_difference = round((df['PO Price'][i] - df['Ti-Cash'][i]) / df['PO Price'][i], 2)
        else:
            relative_difference = 0
    except ZeroDivisionError:
        relative_difference = 0
    
    CM.append(relative_difference)
df['Cash Margin '] = CM

#Calculating the credit margin for each entry in the DataFrame
CRM = []

for i in range(len(df)):
    try:
        if df['PO Price'][i] != 0 and not np.isnan(df['PO Price'][i]):
            relative_difference = round((df['PO Price'][i] - df['Ti-Credits'][i]) / df['PO Price'][i], 2)
        else:
            relative_difference = 0
    except ZeroDivisionError:
        relative_difference = 0
    
    CRM.append(relative_difference)
    
df['Credit Margin '] = CRM

##Calculating the 'Day to release' for each entry in the DataFrame
DTR = []

for i in range(len(df)):
    if df['Pre Order'][i] == 'Yes':
        days_difference = (df['Release Date'][i]- today_date).days
        DTR.append(days_difference)      
    else:
        DTR.append('NA')    
        
df['Days to Release'] = DTR        



df.to_csv('classification.csv')
